In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
#sampleSubmission = pd.read_csv("../input/diabetic-retinopathy-detection/sampleSubmission.csv")
trainLabels = pd.read_csv("../input/diabetic-retinopathy-detection/trainLabels.csv")

In [ ]:
trainLabels.head()

In [ ]:
len(trainLabels)

In [ ]:
from PIL import Image
from keras.preprocessing import image
import os
import numpy as np

# resize the image to (256, 256)
img_rows, img_cols = 256, 256

listing = os.listdir("../input/diabetic-retinopathy-detection/") 
listing.remove("trainLabels.csv")

immatrix = []
imlabel = []
demo_img0 
for file in listing:
    base = os.path.basename("../input/diabetic-retinopathy-detection/" + file)
    fileName = os.path.splitext(base)[0]
    imlabel.append(trainLabels.loc[trainLabels.image==fileName, 'level'].values[0])
    im = Image.open("../input/diabetic-retinopathy-detection/" + file)
    img = np.array(im.resize((img_rows,img_cols)))
    
    # convert to green channel only
    img[:,:,[0,2]] = 0
    immatrix.append(img)

In [ ]:
len(immatrix)

In [ ]:
im = Image.fromarray(immatrix[1],'RGB')
print("level:",imlabel[1])
im

In [ ]:
import random

# define transformation methods
def horizontal_flip(image_array):
    return image_array[:, ::-1]

def vertical_flip(image_array):
    return image_array[::-1,:]

def random_transform(image_array):
    if random.random() < 0.5:
        return vertical_flip(image_array)
    else:
        return horizontal_flip(image_array)

In [ ]:
im = Image.fromarray(vertical_flip(immatrix[1]),'RGB')
im

In [ ]:
length = len(immatrix)
for i in range(length):
    if random.random() < 0.1:
        immatrix.append(random_transform(immatrix[i]))
        imlabel.append(imlabel[i])
        
print("Size of image array before augmentation: ", length)
print("Size fo image array after augmentation: ", len(immatrix))

In [ ]:
from sklearn.utils import shuffle

data,label = shuffle(immatrix, imlabel, random_state=42)
train_data = [data,label]

In [ ]:
import matplotlib.pyplot as plt

plt.hist(label)
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(train_data[0], train_data[1], test_size = 0.1, random_state = 42)

print(np.array(x_train).shape)
print(np.array(y_train).shape)

In [ ]:
from keras.utils import np_utils

y_train = np_utils.to_categorical(np.array(y_train), 5)
y_test = np_utils.to_categorical(np.array(y_test), 5)

x_train = np.array(x_train).astype("float32")/255.
x_test = np.array(x_test).astype("float32")/255.

print(np.array(y_train).shape)

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization

model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=x_train[0].shape))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D())
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D())
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D())
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(activation='softmax', units=5))

model.compile(loss='categorical_crossentropy', optimizer = keras.optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])

In [ ]:
model.summary()

In [ ]:
model.fit(x_train, y_train, batch_size = 64, epochs=10, shuffle=True, verbose=2)

In [ ]:
predictions = model.predict(x_test)
predictions

In [ ]:
score = model.evaluate(x_test, y_test)
print(score)

In [ ]:
from PIL import Image
im = Image.open("../input/diabetic-retinopathy-detection/851_left.jpeg")
im

In [ ]:
img = np.array(im)
img

In [ ]:
im = Image.fromarray(immatrix[1],'RGB')
print("level:",imlabel[1])
im

In [ ]:
test = model.predict(np.array([immatrix[1]]).astype("float32")/255.)
test

In [ ]:
#confusion matrix
from keras import metrics
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels

matrix = confusion_matrix(y_test.argmax(axis=1), predictions.argmax(axis=1))

In [ ]:
matrix